# data split

In [1]:
import os
import argparse
import time
import random
import shutil

from pathlib import Path
from math import ceil

from somenlp.utils import get_time_marker
from somenlp.feature_engineering import calculate_features_parallel


import os
from os import listdir 
from collections import Counter

import os
from os import listdir 
from collections import Counter


in_path = '/home/beck/Desktop/SoMeNLP/data/PLoS_methods_bio/'
out_path = '/home/beck/Desktop/SoMeNLP/data/PLoS_methods/'

def split_data(in_path, out_path, seed_, file_ext =".data.txt", 
               ratio =[60, 20, 20], set_names = ['train', 'devel', 'test'], 
               ):
    
    in_path = in_path.rstrip('/')
    out_path = out_path.rstrip('/')

    if not os.path.isdir(in_path):
        raise(RuntimeError("Input path does not exist"))
        
    if not os.path.isdir(out_path):
        os.mkdir(out_path)
    
    if sum(ratio) != 100:
        raise(RuntimeError("Input ratio {} does not sum to 100".format(ratio)))
    if len(ratio) != len(set_names):
        raise(RuntimeError("Number of ratios and setnames has to match: {} vs {}".format(ratio, set_names)))

    #print("Loading files")
    single_files = list(Path(in_path).rglob('*{}'.format(file_ext)))
    
    all_files = []
    for entry in single_files:
        
        base_file_name = entry.name.split(file_ext)[0]
        
        base_file_entries = list(Path(in_path).rglob('{}*'.format(base_file_name)))
        
        all_files.append(base_file_entries)
    
    random.seed(seed_)
    random.shuffle(all_files)

    #print("Copying files")
    cut_sum = 0
    prev_cut_idx = 0
    for cut, name in zip(ratio, set_names):
        cut_sum += cut
        cut_idx = ceil(len(all_files) * cut_sum / 100) 
        
        out_folder_name = in_path.rsplit('/')[-1]
        new_output_location = '{}/{}_{}'.format(out_path.rstrip('/'), out_folder_name, name) 
        
        if not os.path.isdir(new_output_location):
            os.makedirs(new_output_location)
            
        for files in all_files[prev_cut_idx:cut_idx]:
            for f in files:
                source_path = str(f)
                target_path = '{}/{}'.format(new_output_location, f.name)
                shutil.copy(source_path, target_path)
        prev_cut_idx = cut_idx
    
    print("Done")
    

def purposeLabel_counter(path):
    
    import os
    from os import listdir 
    from collections import Counter
    
    
    def list_file_names(path, file_ext='.labels.txt'):
    
        file_names_list = []

        for file_name in os.listdir(path):
            if not file_name.endswith(file_ext): continue
            file_names_list.append(file_name) 

        file_names_list.sort()

        return file_names_list
    
    file_name_list = list_file_names(path)
    
    interest_list = ["Analysis", "Modelling", "Stimulation", "DataCollection", "DataPreProcss", 
                 "Simulation", "Visualization", "Programming"]
    
    all_purpose_labels = []
    
    for file_name in file_name_list[:]:
        file_path = path + file_name
        
        with open(file_path, 'r') as f:
            
            list_lines = f.readlines()
            
            #print(list_of_lebs)
            list_of_tokens = ' '.join(list_lines).split()
            
            for tok in list_of_tokens:
                if tok != 'O' and (len(tok.split('-')[1].split('_')) == 3 ):
                    if tok.split('-')[1].split('_')[2] in interest_list:
                        
                        purpose = tok.split('-')[1].split('_')[2]
                        
                        all_purpose_labels.append(purpose)
    
                        
    return  dict(Counter(all_purpose_labels))

def train_test_dev(whole_path, train_path, test_path, dev_path):
    
    _whole = purposeLabel_counter(whole_path)
    _train = purposeLabel_counter(train_path)
    _test  = purposeLabel_counter(test_path)
    _dev   = purposeLabel_counter(dev_path)
    
    
    
    _whole_sorted = dict(sorted(_whole.items()))
    _train_sorted = dict(sorted(_train.items()))
    _test_sorted  = dict(sorted(_test.items()))
    _dev_sorted   = dict(sorted(_dev.items()))
    
    return _whole_sorted, _train_sorted, _test_sorted, _dev_sorted

def __proportion_calculator_with_key(whole_dict, train_dict, test_dict, dev_dict ):
    
    train_purpose_props = []
    for (k,v), (k2,v2) in zip(whole_dict.items(), train_dict.items()):
        if k2 == k: 
            train_percentage = int(v2 / v * 100)
            train_purpose_props.append((train_percentage, k))
            
    dev_purpose_props = []
    for (k,v), (k2,v2) in zip(whole_dict.items(), dev_dict.items()):
        if k2 == k:
            
            dev_percentage = int(v2 / v * 100)
            dev_purpose_props.append((dev_percentage,k))
            
    test_purpose_props = []       
    for (k,v), (k2,v2) in zip(whole_dict.items(), test_dict.items()):
        if k2 == k:
            
            test_percentage = int(v2 / v * 100)
            test_purpose_props.append((test_percentage, k))
            
    return train_purpose_props, dev_purpose_props, test_purpose_props


def __proportion_calculator(whole_dict, train_dict, test_dict, dev_dict ):
    
    train_purpose_props = []
    for (k,v), (k2,v2) in zip(whole_dict.items(), train_dict.items()):
        if k2 == k: 
            train_percentage = int(v2 / v * 100)
            train_purpose_props.append(train_percentage)
            
    dev_purpose_props = []
    for (k,v), (k2,v2) in zip(whole_dict.items(), dev_dict.items()):
        if k2 == k:
            
            dev_percentage = int(v2 / v * 100)
            dev_purpose_props.append(dev_percentage)
            
    test_purpose_props = []       
    for (k,v), (k2,v2) in zip(whole_dict.items(), test_dict.items()):
        if k2 == k:
            
            test_percentage = int(v2 / v * 100)
            test_purpose_props.append(test_percentage)
            
    return train_purpose_props, dev_purpose_props, test_purpose_props



def _within_range(list_):
    
    bool_list = []
    for val in list_:
        if val in range(55,65):
            bool_list.append(True)
        else:
            bool_list.append(False)
            
    return all(bool_list) #, bool_list   # apply & to all and return


def _within_range_pubmed(list_):
    
    bool_list = []
    for val in list_:
        if val in range(50,70):
            bool_list.append(True)
        else:
            bool_list.append(False)
            
            
    withn_range_bool = all(bool_list) # apply & to all and return
    
    # check if all class labels are included , class label stimulation is missing in the PubMed 
    if (len(bool_list) == 7):
        all_classes_included = True
    else:
        all_classes_included = False
        
    result = all_classes_included and withn_range_bool
            
    return result   


def _within_range_dev_test(list_):
    
    bool_list = []
    for val in list_:
        if val in range(15,25):
            bool_list.append(True)
        else:
            bool_list.append(False)
            
    return all(bool_list) #, bool_list   # apply & to all and return

In [2]:

# seed the pseudorandom number generator
from random import seed
from random import random

seed(10390)
print(random())

0.028702337771220554


# plos_pubmed_combined

In [4]:
# combined plos pubmed bio   train , test , dev


in_path_comb = '/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined_bio/'
out_path_comb = '/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined/'


i = 10390

split_data(in_path_comb, out_path_comb, i)


comb_path_whole = '/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined_bio/'
comb_path_train = "/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined/plos_pubmed_combined_bio_train/" 
comb_path_test = "/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined/plos_pubmed_combined_bio_test/" 
comb_path_dev = "/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined/plos_pubmed_combined_bio_devel/"
    
    
#calculate the distribution of class_labels for cmbined set
comb_whole_dict, comb_train_dict, comb_test_dict, comb_dev_dict = train_test_dev(comb_path_whole, 
                                                                                 comb_path_train, 
                                                                                 comb_path_test, 
                                                                                 comb_path_dev)


# calculate the proportion for each class label (8 - purposes)
comb_train_props, comb_dev_props, comb_test_props = __proportion_calculator(comb_whole_dict, 
                                                                            comb_train_dict, 
                                                                            comb_test_dict, 
                                                                            comb_dev_dict)


comb_train_result_ = _within_range_pubmed(comb_train_props)
comb_test_result_  = _within_range_dev_test(comb_test_props)
comb_dev_result_   = _within_range_dev_test(comb_dev_props)

print(f'plos+pubmed train {comb_train_props, _within_range(comb_train_props)}')
print(f'plos+pubmed test  {comb_test_props, _within_range_dev_test(comb_test_props)}')
print(f'plos+pubmed dev   {comb_dev_props, _within_range_dev_test(comb_dev_props)}')
print()

print(f"plos+pubmed train within +/- 60% ? {_within_range(comb_train_props)}")
print(f"plos+pubmed test  within +/- 15% ? {_within_range_dev_test(comb_test_props)}")
print(f"plos+pubmed dev   within +/- 15% ? {_within_range_dev_test(comb_dev_props)}")
print()

# train percentage 60%
    
for (k,v), (k2,v2) in zip(comb_whole_dict.items(),  comb_train_dict.items()):
    if k2 == k:
        print(f"{k2 :{18}}  {v2}/{v} train {v2/v*100: .2f}% ")

# test percentage 20% 
print()

for (k,v), (k2,v2) in zip(comb_whole_dict.items(),  comb_test_dict.items()):
    if k2 == k:
        print(f"{k2 :{18}}  {v2}/{v} test {v2/v*100: .2f}% ")
        
# dev percentage 20% 
print()

for (k,v), (k2,v2) in zip(comb_whole_dict.items(),  comb_dev_dict.items()):
    if k2 == k:
        print(f"{k2 :{18}}  {v2}/{v} dev {v2/v*100: .2f}% ")

Done
plos+pubmed train ([61, 60, 61, 62, 63, 57, 63, 59], True)
plos+pubmed test  ([19, 19, 20, 22, 18, 21, 23, 17], True)
plos+pubmed dev   ([18, 19, 18, 15, 17, 21, 12, 23], False)

plos+pubmed train within +/- 60% ? True
plos+pubmed test  within +/- 15% ? True
plos+pubmed dev   within +/- 15% ? False

Analysis            890/1436 train  61.98% 
DataCollection      150/246 train  60.98% 
DataPreProcss       205/335 train  61.19% 
Modelling           141/227 train  62.11% 
Programming         51/80 train  63.75% 
Simulation          16/28 train  57.14% 
Stimulation         45/71 train  63.38% 
Visualization       70/117 train  59.83% 

Analysis            286/1436 test  19.92% 
DataCollection      48/246 test  19.51% 
DataPreProcss       67/335 test  20.00% 
Modelling           51/227 test  22.47% 
Programming         15/80 test  18.75% 
Simulation          6/28 test  21.43% 
Stimulation         17/71 test  23.94% 
Visualization       20/117 test  17.09% 

Analysis            260/1436

In [2]:
# combined plos pubmed bio   train , test , dev


in_path_comb = '/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined_bio/'
out_path_comb = '/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined/'

stable_split_seeds = []

i = 0

split_data(in_path_comb, out_path_comb, i)


comb_path_whole = '/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined_bio/'
comb_path_train = "/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined/plos_pubmed_combined_bio_train/" 
comb_path_test = "/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined/plos_pubmed_combined_bio_test/" 
comb_path_dev = "/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined/plos_pubmed_combined_bio_devel/"
    
    
#calculate the distribution of class_labels for cmbined set
comb_whole_dict, comb_train_dict, comb_test_dict, comb_dev_dict = train_test_dev(comb_path_whole, 
                                                                                 comb_path_train, 
                                                                                 comb_path_test, 
                                                                                 comb_path_dev)


# calculate the proportion for each class label (8 - purposes)
comb_train_props, comb_dev_props, comb_test_props = __proportion_calculator(comb_whole_dict, 
                                                                            comb_train_dict, 
                                                                            comb_test_dict, 
                                                                            comb_dev_dict)


comb_train_result_ = _within_range_pubmed(comb_train_props)
comb_test_result_  = _within_range_dev_test(comb_test_props)
comb_dev_result_   = _within_range_dev_test(comb_dev_props)

bool_ = (comb_train_result_) and (comb_test_result_) and (comb_dev_result_)



while not(bool_):
    
    print(f'splitting ... with seed: {i}')
    
    split_data(in_path_comb, out_path_comb, i)
    
    i += 1
    
    
    
    #calculate the distribution of class_labels for cmbined set
    comb_whole_dict, comb_train_dict, comb_test_dict, comb_dev_dict = train_test_dev(comb_path_whole, 
                                                                                 comb_path_train, 
                                                                                 comb_path_test, 
                                                                                 comb_path_dev)


    # calculate the proportion for each class label (8 - purposes)
    comb_train_props, comb_dev_props, comb_test_props = __proportion_calculator(comb_whole_dict, 
                                                                            comb_train_dict, 
                                                                            comb_test_dict, 
                                                                            comb_dev_dict)
    #print('is the split fair ?')
    comb_train_result_ = _within_range(comb_train_props)
    comb_test_result  = _within_range_dev_test(comb_test_props)
    comb_dev_result   = _within_range_dev_test(comb_dev_props)
    print()
    
    bool_ = (comb_train_result_) and (comb_test_result_) and (comb_dev_result_)
    

    
    
    if bool_:
        print()
        print('plos_pubmed split is balanced. stopped splitting')
        print(f'result:  {comb_train_result_, comb_test_result, comb_dev_result}') 
        stable_split_seeds.append(i)
        break
        
    else:
        
        shutil.rmtree(comb_path_train, ignore_errors=False, onerror=None)
        shutil.rmtree(comb_path_test, ignore_errors=False, onerror=None)
        shutil.rmtree(comb_path_dev, ignore_errors=False, onerror=None)
        
        print(f'result:  {comb_train_result_, comb_test_result, comb_dev_result}')   
        print(f'plos_pubmed split is not yet balanced {comb_train_props, comb_dev_props, comb_test_props}.  splitting further ...')
        print()    

Done
splitting ... with seed: 0
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([64, 56, 65, 67, 77, 50, 60, 54], [19, 24, 19, 18, 17, 25, 21, 22], [16, 18, 15, 13, 5, 25, 18, 23]).  splitting further ...

splitting ... with seed: 1
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([62, 59, 62, 57, 53, 78, 53, 58], [20, 21, 21, 22, 12, 21, 11, 22], [16, 18, 16, 19, 33]).  splitting further ...

splitting ... with seed: 2
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([55, 62, 47, 51, 62, 67, 50, 65], [21, 19, 27, 28, 7, 17, 45, 21], [22, 17, 25, 20, 30, 14, 4, 12]).  splitting further ...

splitting ... with seed: 3
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([58, 59, 60, 62, 67, 75, 64, 62], [21, 17, 20, 26, 11, 7, 21, 20], [20, 23, 19, 10, 21, 17, 14, 17]).  splitting further ...

splitting ... with seed: 4
Done

result:  (False, False, False)
plos_pubmed split is not ye

KeyboardInterrupt: 

In [ ]:
in_path_comb = '/home/beck/Desktop/Otimal_split/plos_pubmed_combined_bio/'
out_path_comb = '/home/beck/Desktop/Otimal_split/plos_pubmed_combined/'

#split_data(in_path_comb, out_path_comb, 204)


comb_path_whole = '/home/beck/Desktop/Otimal_split/plos_pubmed_combined_bio/'
comb_path_train = "/home/beck/Desktop/Otimal_split/plos_pubmed_combined/plos_pubmed_combined_bio_train/" 
comb_path_test = "/home/beck/Desktop/Otimal_split/plos_pubmed_combined/plos_pubmed_combined_bio_test/" 
comb_path_dev = "/home/beck/Desktop/Otimal_split/plos_pubmed_combined/plos_pubmed_combined_bio_devel/"
    
    
#calculate the distribution of class_labels for cmbined set
comb_whole_dict, comb_train_dict, comb_test_dict, comb_dev_dict = train_test_dev(comb_path_whole, 
                                                                                 comb_path_train, 
                                                                                 comb_path_test, 
                                                                                 comb_path_dev)


# calculate the proportion for each class label (8 - purposes)
comb_train_props, comb_dev_props, comb_test_props = __proportion_calculator(comb_whole_dict, 
                                                                            comb_train_dict, 
                                                                            comb_test_dict, 
                                                                            comb_dev_dict)

comb_train_result_ = _within_range_pubmed(comb_train_props)
comb_test_result_  = _within_range_dev_test(comb_test_props)
comb_dev_result_   = _within_range_dev_test(comb_dev_props)


print(f'plos+pubmed train {comb_train_props, _within_range(comb_train_props)[1]}')
print(f'plos+pubmed test  {comb_test_props, _within_range_dev_test(comb_test_props)[1]}')
print(f'plos+pubmed dev   {comb_dev_props, _within_range_dev_test(comb_dev_props)[1]}')
print()

print(f"plos+pubmed train within +/- 60% ? {_within_range(comb_train_props)[0]}")
print(f"plos+pubmed test  within +/- 15% ? {_within_range_dev_test(comb_test_props)[0]}")
print(f"plos+pubmed dev   within +/- 15% ? {_within_range_dev_test(comb_dev_props)[0]}")
print()

# train percentage 60%
    
for (k,v), (k2,v2) in zip(comb_whole_dict.items(),  comb_train_dict.items()):
    if k2 == k:
        print(f"{k2 :{18}}  {v2}/{v} train {v2/v*100: .2f}% ")

# test percentage 20% 
print()

for (k,v), (k2,v2) in zip(comb_whole_dict.items(),  comb_test_dict.items()):
    if k2 == k:
        print(f"{k2 :{18}}  {v2}/{v} test {v2/v*100: .2f}% ")
        
# dev percentage 20% 
print()

for (k,v), (k2,v2) in zip(comb_whole_dict.items(),  comb_dev_dict.items()):
    if k2 == k:
        print(f"{k2 :{18}}  {v2}/{v} dev {v2/v*100: .2f}% ")